# ndarry Object Internals

The NumPy ndarry provies a means to interpret a block of homogeneous data (either contiguous or strided, more on this later) as a multidimensional array object. As you've seen, the data type, or dtype, dtermines how the data is interpreted as being floating point, integer, boolean, or any of the other types we've been looking at.

Part of what makes ndarry powerful is that every array object is strided view on a block of data. You might wonder, for example, how the array view arr[::2, ::-1] does not copy any data. Simply put, the ndarray is more than just a chunk of memory and a dtype; it also has striding information which enables the array to move through memory with varying step sizes. More precisely, the ndarray internally consists of the following:

- A pointer to data, that is a block of system memory
- The data type or dtype
- A tuple indicating the array's shape; For example, a 10 by 5 array would have shape (10, 5)

        np.ones((10, 5).shape)
        (10, 5)
- a tuple of strides, integers indicating the number of bytes to 'step' in order to advance one element along a dimension; For example, a typical (C order) 3 x 4 x 5 array of float64 (8-byte) values have strides (160, 40, 8)

        np.ones((3, 4, 5), dtype = np.float64).strides
        (160, 40, 8)

While it is rare that a typical NumPy user would be interested in the array strides, they are the critical ingredient in constructing copyless array views. Strides can even be negative which enables an array to move backward through memory, which yould be the case in a slice like

        obj[::-1] or obj[:, ::-1]

![The NumPy ndarray object](../../Pictures/The%20NumPy%20ndarray%20object.png)

## NumPy dtype Hierarchy

You may occasionally have code which needs to check whether an array contains integers, floating point numbers, strings, or Python objects. Because there are many types of floating point numbers (float16 through float128), checking that the dtype is among a list of types would be very verbose. Fortunately, the dtypes have superclasses such as np.integer and np.floating which can be used in conjunction with the np.issubd type function:

In [2]:
import numpy as np

In [3]:
ints = np.ones(10, dtype=np.uint16)

floats = np.ones(10, dtype=np.float32)

floats = np.ones(10, dtype=np.float32)

np.issubdtype(floats.dtype, np.floating)

True

In [4]:
np.issubdtype(ints.dtype, np.integer)

True

In [5]:
ints

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=uint16)

You can see all of the parent classes of a specific dtype by calling the type's mro method:

In [6]:
np.float64.mro()

[numpy.float64,
 numpy.floating,
 numpy.inexact,
 numpy.number,
 numpy.generic,
 float,
 object]

![image.png](../../Pictures/The%20NumPy%20dtype%20class%20hierarchy.png)

## Advanced Array Manipulation

There are many ways to work with arrays beyond fancy indexing, slicing, and boolean subsetting. While much of the heavy lifting for data analysis applications is handled by higher level functions in pandas, you may at some point need to write a data algorithm that is not found in one of the existing libraries.

### Reshaping Arrays

Given what we know about NumPy arrays, it should come as little surprise that you can convert an array from one shape to another without copying any data. To do this, pass a tuple indicating the new shape to the reshape array instance method. For example, suppose we had a one-dimensional array of values that we wished to rearrange into a matrix:

In [7]:
arr = np.arange(8)

arr

array([0, 1, 2, 3, 4, 5, 6, 7])

In [8]:
arr.reshape(4,2)

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

> A mulitdimesionnal array can also be reshaped:

In [9]:
arr.reshape(4,2).reshape(2,4)

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

One of the passed shape dimensions can be -1, in which case the value used for that dimension will be inferred from the data:

In [10]:
arr = np.arange(15)

arr

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [11]:
arr.reshape(5, -1) # 1 is minus for 15

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [12]:
other_arr = np.ones((3, 5))

other_arr.shape

(3, 5)

In [13]:
arr.reshape(other_arr.shape)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

he opposite operation of reshape from one-dimensional to a higher dimension is typically known as flattening or raveling:

In [14]:
arr = np.arange(15).reshape((5, 3))

arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [15]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

ravel does not produce a copy of the underlying data if it does not have to (more on this below). The flatten method behaves like ravel except it always returns a copy of the data:

In [16]:
arr.flatten()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

The data can be reshaped or raveled in different orders. This is a slightly nuanced topic for new NumPy users and is therefore the next subtopic.

## C versus Fortran Order

Contrary to some other scientific computing environments like R and MATLAB, NumPy gives you much more control and flexibility over the layout of your data in memory. By default, NumPy arrays are created in row major order. Spatially this means that if you have a two-dimensional array of data, the items in each row of the array are stored in adjacent memory locations. The alternative to row major ordering is column major order, which means that (you guessed it) values within each column of data are stored in adjacent memory locations.

For historical reasons, row and column major order are also know as C and Fortran order, respectively. In FORTRAN 77, the language of our forebears, matrices were all column major.

Functions like reshape and ravel, accept an order argument indicating the order to use the data in the array. This can be 'C' or 'F' in most cases (there are also less commonly-used options 'A' and 'K'; see the NumPy documentation).

In [17]:
arr = np.arange(12).reshape((3,4))

arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [18]:
arr.ravel()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [19]:
arr.ravel('F')

array([ 0,  4,  8,  1,  5,  9,  2,  6, 10,  3,  7, 11])

Reshaping arrays with more than two dimensions can be a bit mind-bending. The key difference between C and Fortran order is the order in which the dimensions are walked:

    C / row major order: traverse higher dimensions first (e.g. axis 1 before advancing on axis 0).

    Fortran / column major order: traverse higher dimensions last (e.g. axis 0 before advancing on axis 1).

![Reshaping in C(row major) or Fortran(column major) order](../../Pictures/Reshaping%20in%20C(row%20major)%20or%20Fortran(column%20major)%20order.png)

### Concatenating and Splitting Arrays

numpy.concatenate takes a sequence (tuple, list, etc.) of arrays and joins them together in order along the input axis.

In [20]:
arr1 = np.array([[1,2,3], [4,5,6]])

arr2 = np.array([[7, 8, 9], [10, 11, 12]])

np.concatenate([arr1, arr2], axis = 0), np.concatenate([arr1, arr2], axis = 1)

(array([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]]),
 array([[ 1,  2,  3,  7,  8,  9],
        [ 4,  5,  6, 10, 11, 12]]))

In [21]:
np.vstack((arr1, arr2)), np.hstack((arr1, arr2))

(array([[ 1,  2,  3],
        [ 4,  5,  6],
        [ 7,  8,  9],
        [10, 11, 12]]),
 array([[ 1,  2,  3,  7,  8,  9],
        [ 4,  5,  6, 10, 11, 12]]))

split, on the other hand, slices apart an array into multiple arrays along an axis:

In [22]:
from numpy.random import randn

In [23]:
arr = randn(5, 2)

arr

array([[-1.21521209,  0.85262362],
       [-0.42887778, -0.52856777],
       [-1.00735231,  0.44110195],
       [ 0.93593244, -1.21236712],
       [ 0.0492094 , -0.04855738]])

In [24]:
first, second, third = np.split(arr, [1, 3])

In [25]:
first, second, third

(array([[-1.21521209,  0.85262362]]),
 array([[-0.42887778, -0.52856777],
        [-1.00735231,  0.44110195]]),
 array([[ 0.93593244, -1.21236712],
        [ 0.0492094 , -0.04855738]]))

In [26]:
arr = np.arange(14).reshape(7,2)

arr

array([[ 0,  1],
       [ 2,  3],
       [ 4,  5],
       [ 6,  7],
       [ 8,  9],
       [10, 11],
       [12, 13]])

In [27]:
aa, bb, cc, dd = np.split(arr, [3, 5, 7])

aa, bb, cc

(array([[0, 1],
        [2, 3],
        [4, 5]]),
 array([[6, 7],
        [8, 9]]),
 array([[10, 11],
        [12, 13]]))

![Array concatenations functions](../../Pictures/Array%20concatenations%20functions.png)

#### Stacking helpers: r_ and c_

There are two special objects in the NumPy namespace, r_ and c_, that make stacking arrays more concise:

In [28]:
arr = np.arange(6)

arr1 = arr.reshape(3,2)

arr2 = randn(3,2)

In [29]:
np.r_[arr1, arr2]

array([[ 0.        ,  1.        ],
       [ 2.        ,  3.        ],
       [ 4.        ,  5.        ],
       [ 0.02109909,  2.11270202],
       [ 0.53535766, -0.12003113],
       [ 0.85898445,  0.243189  ]])

In [30]:
np.c_[np.r_[arr1, arr2], arr]

array([[ 0.        ,  1.        ,  0.        ],
       [ 2.        ,  3.        ,  1.        ],
       [ 4.        ,  5.        ,  2.        ],
       [ 0.02109909,  2.11270202,  3.        ],
       [ 0.53535766, -0.12003113,  4.        ],
       [ 0.85898445,  0.243189  ,  5.        ]])

> These additionally can translate slices to arrays:

In [31]:
np.c_[1:6, -10:-5]

array([[  1, -10],
       [  2,  -9],
       [  3,  -8],
       [  4,  -7],
       [  5,  -6]])

### Repeating Elements: Tile and Repeat

The two main tools for repeating or replicating arrays to produce larger arrays are the  repeat and tile functions. repeat replicates each element in an array some number of times, producing a larger array:

In [32]:
arr = np.arange(3)

arr.repeat(3)

array([0, 0, 0, 1, 1, 1, 2, 2, 2])

By default, if you pass an integer, each element will be repeated that number of times. If you pass an array of integers, each element can be repeated a different number of times:

In [33]:
arr.repeat([15, 3, 4])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2])

Multidimensional arrays can have their elements repeated along a particular axis.

In [34]:
arr = randn(2, 2)

arr

array([[0.56765919, 1.15921044],
       [1.1336333 , 1.21550627]])

In [35]:
arr.repeat(2, axis = 0)

array([[0.56765919, 1.15921044],
       [0.56765919, 1.15921044],
       [1.1336333 , 1.21550627],
       [1.1336333 , 1.21550627]])

Note that if no axis is passed, the array will be flattened first, which is likely not what you want. Similarly you can pass an array of integers when repeating a multidimensional array to repeat a given slice a different number of times:

In [36]:
arr.repeat([2, 3], axis = 1)

array([[0.56765919, 0.56765919, 1.15921044, 1.15921044, 1.15921044],
       [1.1336333 , 1.1336333 , 1.21550627, 1.21550627, 1.21550627]])

tile, on the other hand, is a shortcut for stacking copies of an array along an axis. You can visually think about it as like “laying down tiles”:

In [37]:
arr = np.arange(15).reshape(5, -1)

arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [38]:
np.tile(arr, 3)

array([[ 0,  1,  2,  0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14, 12, 13, 14]])

The second argument is the number of tiles; with a scalar, the tiling is made row-byrow, rather than column by column: The second argument to tile can be a tuple indicating the layout of the “tiling”:

In [39]:
arr

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11],
       [12, 13, 14]])

In [40]:
np.tile(arr, (2,2))

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14],
       [ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14]])

In [41]:
np.tile(arr, (3, 2))

array([[ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14],
       [ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14],
       [ 0,  1,  2,  0,  1,  2],
       [ 3,  4,  5,  3,  4,  5],
       [ 6,  7,  8,  6,  7,  8],
       [ 9, 10, 11,  9, 10, 11],
       [12, 13, 14, 12, 13, 14]])

### Fancy Indexing Equivalents: Take and Put

As you may recall from Chapter 4, one way to get and set subsets of arrays is by  fancy indexing using integer arrays:

In [42]:
arr = np.arange(10) *100

In [43]:
inds = [7, 1, 2, 6]

In [44]:
arr[inds]

array([700, 100, 200, 600])

There are alternate ndarray methods that are useful in the special case of only making a selection on a single axis:

In [45]:
arr.take(inds)

array([700, 100, 200, 600])

In [46]:
arr.put(inds, 11)

arr

array([  0,  11,  11, 300, 400, 500,  11,  11, 800, 900])

In [47]:
arr.put(inds, [1,2,3,4,5,6,7])

arr

array([  0,   2,   3, 300, 400, 500,   4,   1, 800, 900])

In [48]:
inds = [2, 0, 2, 1]

arr = randn(2, 4)

arr

array([[ 2.04732615, -0.98320342,  0.66435187, -1.26833812],
       [ 2.09929607, -0.17653928, -1.29727812,  0.06464323]])

In [49]:
arr.take(inds, axis = 1)

array([[ 0.66435187,  2.04732615,  0.66435187, -0.98320342],
       [-1.29727812,  2.09929607, -1.29727812, -0.17653928]])

In [50]:
arr = randn(1000, 50)

# Random sample of 500 rows

inds = np.random.permutation(1000)[:500]

In [51]:
%timeit arr[inds]

45 µs ± 2.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [52]:
%timeit arr.take(inds, axis = 0)

35.2 µs ± 5.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [53]:
print('a {0} b {1} {2}'.format('aa', 'bb', 'cc'))

a aa b bb cc


In [54]:
data = [1,2,3,4,5]
data.pop()
data


[1, 2, 3, 4]

In [55]:
2**(3**2), (2**3)**2, (2**3)**3

(512, 64, 512)

In [56]:
a = set('abc')
a.add('san')
a.update(set(['p', 'o']))

a

{'a', 'b', 'c', 'o', 'p', 'san'}

## Broadcasting

Broadcasting describes how arithmetic works between arrays of differnet shapes. It is a very powerful feature, but one that can be easily misunderstood, even by experienced users. The simplest example of broadcasting occures when combining a scalar value with an array:

In [57]:
arr = np.arange(5)

arr * 4

array([ 0,  4,  8, 12, 16])

Here we say that the scalar value 4 has been broadcast to all of the other elements in the multiplications operation.

For example, we can demean each column of an array by subtracting the column means.

In this case, it is very simple:

In [58]:
arr = np.random.randn(4, 3)

In [59]:
arr.mean(0)

array([ 0.38168484, -0.54935974, -0.32307556])

In [60]:
demaned = arr - arr.mean(0)

In [61]:
demaned, demaned.mean()

(array([[-0.42631053, -0.05009524, -0.37134791],
        [ 0.48061332, -0.93832601, -0.6568426 ],
        [-0.47813633,  0.75833098,  1.56990567],
        [ 0.42383354,  0.23009027, -0.54171517]]),
 0.0)

![Broadcasting over axis 0 with a 1D array](../../Pictures/Broadcasting%20over%20axis%200%20with%20a%201D%20array.png)

                The Broadcasting Ru

Two arrays are compatible for broadcasting if for each trailing dimension (that is, starting from the end), the axis lengths match or if either of the lengths is 1. Broadcasting is then performed over the missing and / or length 1 dimensions.

Since arr.mean(0) has length 3, it is compatible for broadcasting across axis 0 because the trailing dimension in arr is 3 and therefore matches. According to the rules, to subtract over axis 1 (that is, subtract the row mean from each row), the smaller array must have shape (4, 1):

In [62]:
arr

array([[-0.0446257 , -0.59945498, -0.69442347],
       [ 0.86229815, -1.48768575, -0.97991816],
       [-0.09645149,  0.20897124,  1.24683011],
       [ 0.80551838, -0.31926946, -0.86479073]])

In [63]:
row_means = arr.mean(1)

In [64]:
row_means, row_means.reshape(4, 1)

(array([-0.44616805, -0.53510192,  0.45311662, -0.12618061]),
 array([[-0.44616805],
        [-0.53510192],
        [ 0.45311662],
        [-0.12618061]]))

In [65]:
demaned = arr - row_means.reshape(4, 1)

In [66]:
demaned.mean(1)

array([ 1.85037171e-17, -3.70074342e-17,  0.00000000e+00,  0.00000000e+00])

![Broadcasting over axis 1 of a 2D array](../../Pictures/Broadcasting%20over%20axis%201%20with%20a%202D%20array.png)

### Broadcasting Over Other Axes

Broadcasting with higher dimensional arrays can seem even more mind-bending, but it is really a matter of following the rules. If you don’t, you’ll get an error like this:

In [67]:
arr - arr.mean(0)

array([[-0.42631053, -0.05009524, -0.37134791],
       [ 0.48061332, -0.93832601, -0.6568426 ],
       [-0.47813633,  0.75833098,  1.56990567],
       [ 0.42383354,  0.23009027, -0.54171517]])

![Broadcasting over axis 0 with a 3D array](../../Pictures/Broadcasting%20over%20axis%200%20with%20a%203D%20array.png)

It’s quite common to want to perform an arithmetic operation with a lower dimensional array across axes other than axis 0. According to the broadcasting rule, the “broadcast dimensions” must be 1 in the smaller array. In the example of row demeaning above this meant reshaping the row means to be shape (4, 1) instead of (4,):

In [68]:
arr - arr.mean(1).reshape(4, 1)

array([[ 0.40154235, -0.15328693, -0.24825542],
       [ 1.39740007, -0.95258383, -0.44481624],
       [-0.54956811, -0.24414538,  0.79371349],
       [ 0.93169898, -0.19308886, -0.73861013]])

A very common problem, therefore, is needing to add a new axis with length 1 specifically for broadcasting purposes, especially in generic algorithms. Using reshape is one option, but inserting an axis requires constructing a tuple indicating the new shape. This can often be a tedious exercise. Thus, NumPy arrays offer a special syntax for inserting new axes by indexing. We use the special np.newaxis attribute along with “full” slices to insert the new axis:

In [69]:
arr = np.zeros ((4, 4))

In [70]:
arr_3d = arr[:, np.newaxis, :]

In [71]:
arr_3d

array([[[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]],

       [[0., 0., 0., 0.]]])

In [72]:
arr_1d = np.random.normal(size = 3)

arr_1d

array([0.84018348, 0.24021514, 0.06001286])

In [73]:
arr_1d[:, np.newaxis]

array([[0.84018348],
       [0.24021514],
       [0.06001286]])

In [74]:
arr_1d[np.newaxis, :]

array([[0.84018348, 0.24021514, 0.06001286]])

![Compatible 2D array shapes for broadcasting over a 3D array](../../Pictures/Compatible%202D%20array%20shapes%20for%20broadcasting%20over%20a%203D%20array.png)

Thus, if we had a three-dimensional array and wanted to demean axis 2, say, we would only need to write:

In [75]:
arr = np.arange(60).reshape(3, 4, 5)

arr

array([[[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15, 16, 17, 18, 19]],

       [[20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34],
        [35, 36, 37, 38, 39]],

       [[40, 41, 42, 43, 44],
        [45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54],
        [55, 56, 57, 58, 59]]])

In [76]:
arr.mean(0),arr.mean(1),arr.mean(2)

(array([[20., 21., 22., 23., 24.],
        [25., 26., 27., 28., 29.],
        [30., 31., 32., 33., 34.],
        [35., 36., 37., 38., 39.]]),
 array([[ 7.5,  8.5,  9.5, 10.5, 11.5],
        [27.5, 28.5, 29.5, 30.5, 31.5],
        [47.5, 48.5, 49.5, 50.5, 51.5]]),
 array([[ 2.,  7., 12., 17.],
        [22., 27., 32., 37.],
        [42., 47., 52., 57.]]))

In [77]:
depth_means = arr.mean(2)

In [78]:
demaned = arr - depth_means[:, :, np.newaxis]

In [79]:
demaned.mean(2)

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

> If you’re completely confused by this, don’t worry. With practice you will get the hang of it!

Some readers might wonder if there’s a way to generalize demeaning over an axis without sacrificing performance. There is, in fact, but it requires some indexing gymnastics:

In [80]:
def demean_axis(arr, axis = 0):
    means = arr.mean(axis)

    # This generalized things like [:, :, np.neawaxis] to N dimensions
    indexer = [slice(None)] * arr.ndim
    indeer[axis] = np.newaxis
    return arr - means[indexer]

### Setting Array Values by Broadcasitn

The same breadcasting rule governing arithmetic operations also applies to seting values via array indexing. In the simplest case, we can do things like:

In [81]:
arr = np.zeros((4, 3))

arr[:] = 5

arr

array([[5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.],
       [5., 5., 5.]])

However, if we had a one-dimensional array of values we wanted to set to set into the volumns of the array, we can do that as long as the shape is compatible:

In [82]:
col = np.array([1.28, -0.42, 0.44, 1.6])

In [83]:
arr[:] = col[:, np.newaxis]

arr

array([[ 1.28,  1.28,  1.28],
       [-0.42, -0.42, -0.42],
       [ 0.44,  0.44,  0.44],
       [ 1.6 ,  1.6 ,  1.6 ]])

In [84]:
arr[:2] = [[-1.37], [0.509]]

arr

array([[-1.37 , -1.37 , -1.37 ],
       [ 0.509,  0.509,  0.509],
       [ 0.44 ,  0.44 ,  0.44 ],
       [ 1.6  ,  1.6  ,  1.6  ]])

## Advanced ufunc Usage

While many NumPy users will only make use of the fast element-wise operations provided by the universal functions, there are a number of additional features that occasionally can help you write more concise code without loops.

### Ufunc Instance Methods

Each of NumPy's binary ufuncs has special methods for performing certain kinds of special vectorized operations. These are summarized, but I'll give a few concrete examples to illustrate how they work.

reduce takes a single array and aggregates its values, optionally a long an axis, by performing a sequence of binary operations. For example, an alternate way to sum elements in an array is to use np.add.reduce:

In [85]:
arr = np.arange(10)

arr

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [86]:
np.add.reduce(arr)

45

In [87]:
arr.sum()

45

The starting value (0 for add) depends on the ufunc. If an axis is passed, the reduction is performed along that axis. This allows you to answer certain kinds of questions in a concise way. As a less trivial example, we can use np.logical_and to check whether the values in each row of an array are sorted:

In [88]:
arr = randn(5, 5)

In [89]:
arr[::2].sort(1) # sort a few rows

arr

array([[-1.35926962,  0.65477371,  0.66555711,  0.80925998,  0.90650711],
       [-0.33655924, -0.06591403, -0.51219828, -0.06237008, -0.02150359],
       [-0.90237471, -0.23169677,  0.24985084,  1.0330736 ,  1.67787467],
       [ 0.47364978, -0.5467195 , -0.01460727,  0.46045581,  1.95707164],
       [-1.79105368, -0.91838016, -0.51082252, -0.32022423,  2.59282052]])

In [90]:
arr[:, :-1]< arr[:, 1:]

array([[ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True],
       [False,  True,  True,  True],
       [ True,  True,  True,  True]])

In [91]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis=1)

array([ True, False,  True, False,  True])

Of course, logical_and.reudce is equivalent to the all method.

accumulate is related to reduce like cumsum is related to sum. It produces an array of the same size with the intermediate “accumulated” values:

In [92]:
arr = np.arange(15).reshape(3, 5)

In [93]:
np.add.accumulate(arr, axis = 1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

outer performs a pairwise cross-product between two arrays:

In [94]:
arr = np.arange(3).repeat([2, 4, 2])

In [95]:
arr

array([0, 0, 1, 1, 1, 1, 2, 2])

In [96]:
np.multiply.outer(arr, np.arange(5))

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 2, 4, 6, 8],
       [0, 2, 4, 6, 8]])

The output of outer will have a dimension that is the sum of the dimensions of the inputs:

In [97]:
result = np.subtract.outer(randn(3, 4), randn(5))

In [98]:
result.shape

(3, 4, 5)

The last method, reduceat, performs a “local reduce”, in essence an array groupby operation in which slices of the array are aggregated together. While it’s less flexible than the GroupBy capabilities in pandas, it can be very fast and powerful in the right circumstances. It accepts a sequence of “bin edges” which indicate how to split and aggregate the values:

In [99]:
arr = np.arange(10)

In [100]:
np.add.reduceat(arr, [1, 0, 2])

array([ 1,  1, 44])

The results are the reductions (here, sums) performed over arr[0:5], arr[5:8], and arr[8:]. Like the other methods, you can pass an axis argument:

In [101]:
arr = np.multiply.outer(np.arange(4), np.arange(5))

In [102]:
arr

array([[ 0,  0,  0,  0,  0],
       [ 0,  1,  2,  3,  4],
       [ 0,  2,  4,  6,  8],
       [ 0,  3,  6,  9, 12]])

In [103]:
np.add.reduceat(arr, [0, 0, 4], axis= 1)

array([[ 0,  0,  0],
       [ 0,  6,  4],
       [ 0, 12,  8],
       [ 0, 18, 12]])

![unfunc methods](../../Pictures/unfunc%20methods.png)

## Custom ufuncs

There are a couple facilities for creating your own functions with ufunc-like semantics. numpy.frompyfuncaccepts a Python functoin along with a specification for the number of inputs and outputs. For example, a simple function that adds element-wise would be specified as:

In [104]:
def add_elements(x, y):
    return x + y

In [105]:
add_them = np.frompyfunc(add_elements, 2, 1)

add_them

<ufunc 'add_elements (vectorized)'>

In [106]:
add_them(np.arange(8), np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

Functions created using frompyfunc always return arrays of Python objects which isn’t very convenient. Fortunately, there is an alternate, but slightly less featureful function  numpy.vectorize that is a bit more intelligent about type inference:

In [107]:
add_them = np.vectorize(add_elements, otypes=[np.float64])

In [108]:
add_them(np.arange(8), np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

These functions provide a way to create ufunc-like functions, but they are very slow because they require a Python function call to compute each element, which is a lot slower than NumPy’s C-based ufunc loops:

In [109]:
arr = randn(10000)

In [110]:
%timeit add_them(arr, arr)

3.28 ms ± 724 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [111]:
%timeit np.add(arr, arr)

7.65 µs ± 118 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


There are a number of projects under way in the scientific Python community to make it easier to define new ufuncs whose performance is closer to that of the built-in ones.

# Structured and Record Arrays

You may have noticed up until now that ndarray is a homogeneous data container; that is, it represents a block of memory in which each element takes up the same number of bytes, determined by the dtype. On the surface, this would appear to not allow you to represent heterogeneous ro tabular-like data. A structured array is an ndarray in which each element can be thought of as representing a struct in C (hence the "structured" name) or a row in a SQL table with multiple named fields:

In [114]:
dty = [('x', np.float64), ('y', np.int32)]

In [116]:
sarr = np.array([(1.5, 6), (np.pi, -1)], dtype = dty)

In [117]:
sarr

array([(1.5       ,  6), (3.14159265, -1)],
      dtype=[('x', '<f8'), ('y', '<i4')])

There are several ways to specify a structured dtype (see the online NumPy documentation). One typical way is as a list of tuples with (field_name, field_data_type). Now, the elements of the array are tuple-like objects whose elements can be accessed like a dictionary:

In [119]:
sarr[0]

(1.5, 6)

In [128]:
sarr[0]['x'], sarr[0]['y']

(1.5, 6)

The field names are stored in the dtype.names attribute. On accessing a field on the structured array, a strided view on the data is returned thus copying nothing:

In [129]:
sarr['x']

array([1.5       , 3.14159265])

## Nested dtypes and Multidimensional Fields

When specifying a structured dtype, you can additionally pass a shape (as an int or tupe):

In [130]:
dtype = [('x', np.int64, 3), ('y', np.int32)]

In [131]:
arr = np.zeros(4, dtype=dtype)

In [132]:
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

In this case, the x field now refers to an array of length three for each record:

In [140]:
arr[0]['x']

array([0, 0, 0], dtype=int64)

Conveniently, accessing arr['x'] then returns a two-dimensional array instead of a one-dimensional array as in prior examples:

In [141]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

This enables you to express more complicated, nested structures as a single block of memory in an array. Though, since dtypes can be arbitrarily complex, why not nested dtypes? Here is a simple example:

In [142]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), ('y', np.int32)]

In [144]:
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype = dtype)

In [146]:
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [152]:
data['y']

array([5, 6])

In [150]:
data['x']['a'], data['x']['b']

(array([1., 3.]), array([2., 4.], dtype=float32))

As you can see, variable-shape fields and nested records is a very rich feature that can be the right tool in certain circumstances. A DataFrame from pandas, by contrast, does not support this feature directly, though it is similar to hierarchical indexing.

# More About Sorting

Like Python's built-in list, the ndarray sort instance method is an in-place sort, meaning that the array contents are rearranged without producing a new array:

In [153]:
arr = randn(6)

In [157]:
arr.sort()

arr

array([-1.43399911, -0.6603304 ,  0.1764409 ,  0.93626598,  0.99496337,
        1.12612437])

When sorting arrays in-place, remember that if the array is a view on a different ndarray, the original array will be modified:

In [158]:
arr = randn(3, 5)

arr

array([[ 1.38417908, -0.1791828 ,  0.12842321,  0.1137134 , -1.12965523],
       [ 0.24993977,  0.32663432, -0.66402213, -1.57754814,  0.68829422],
       [ 1.80283223,  0.82267975, -0.34954649, -0.47243354,  1.55450481]])

In [159]:
arr[:, 0].sort() # sort first column values in-place

In [160]:
arr

array([[ 0.24993977, -0.1791828 ,  0.12842321,  0.1137134 , -1.12965523],
       [ 1.38417908,  0.32663432, -0.66402213, -1.57754814,  0.68829422],
       [ 1.80283223,  0.82267975, -0.34954649, -0.47243354,  1.55450481]])

On the other hand, numpy.sort creates a new, sorted copy of an array. Otherwise it accepts the same arguments (such as kind, more on this below) as ndarray.sort:

In [173]:
arr = randn(5)

arr

array([-0.35510794,  0.13148943,  0.06134748,  0.03044466,  1.5512107 ])

In [172]:
np.sort(arr)

arr

array([ 0.06214953,  1.80623456, -0.1171063 ,  0.40258533,  1.15245917])

All of these sort methods take an axis argument for sorting the sections of data along the passed axis independently:

In [174]:
arr = randn(3, 5)

arr

array([[ 2.18981465, -0.7247121 , -0.55326352,  1.07751642,  0.05117344],
       [ 2.44719202, -1.02024953,  0.43062127, -1.3654896 ,  0.80644997],
       [-0.1176083 ,  0.72971541,  0.7866898 , -0.2037501 , -1.43829067]])

In [175]:
arr.sort(axis = 1)

In [176]:
arr

array([[-0.7247121 , -0.55326352,  0.05117344,  1.07751642,  2.18981465],
       [-1.3654896 , -1.02024953,  0.43062127,  0.80644997,  2.44719202],
       [-1.43829067, -0.2037501 , -0.1176083 ,  0.72971541,  0.7866898 ]])

You may notice that none of the sort methods have an option to sort in descending order. This is not actually a big deal because array slicing produces views, thus not producing a copy or requiring any computational work. Many Python users are familiar with the “trick” that for a list values, values[::-1] returns a list in reverse order. The same is true for ndarrays:

In [185]:
arr [:, ::-1]

array([[ 2.18981465,  1.07751642,  0.05117344, -0.55326352, -0.7247121 ],
       [ 2.44719202,  0.80644997,  0.43062127, -1.02024953, -1.3654896 ],
       [ 0.7866898 ,  0.72971541, -0.1176083 , -0.2037501 , -1.43829067]])

### Indirect Sorts: argsort and lexsort

In data analysis it’s very common to need to reorder data sets by one or more keys. For example, a table of data about some students might need to be sorted by last name then by first name. This is an example of an indirect sort, and if you’ve read the pandasrelated chapters you have already seen many higher-level examples. Given a key or keys (an array or values or multiple arrays of values), you wish to obtain an array of integer indices (I refer to them colloquially as indexers) that tells you how to reorder the data to be in sorted order. The two main methods for this are argsort and numpy.lexsort. As a trivial example:

In [186]:
values = np.array([5, 0, 1, 3, 2])

In [187]:
indexer = values.argsort()

In [188]:
indexer

array([1, 2, 4, 3, 0], dtype=int64)

In [189]:
values[indexer]

array([0, 1, 2, 3, 5])

As a less trivial example, this code reorders a 2D array by its first row:

In [190]:
arr = randn(3, 5)

In [191]:
arr[0] = values

In [192]:
arr

array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
       [ 0.67831472, -0.35238804,  1.25301581,  0.27357961,  0.5348519 ],
       [ 1.17376692, -0.2186713 ,  0.6328656 ,  0.30954104,  0.2833018 ]])

lexsort is similar to argsort, but it performs an indirect lexicographical sort on multiple key arrays. Suppose we wanted to sort some data identified by first and last names:

In [193]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])

last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])

In [194]:
sorter = np.lexsort((first_name, last_name))

In [195]:
sorter

array([1, 2, 3, 0, 4], dtype=int64)

In [211]:
sorted = zip(last_name[sorter], first_name[sorter])

In [213]:
print(set(sorted))

{('Arnold', 'Steve'), ('Arnold', 'Jane'), ('Walters', 'Barbara'), ('Jones', 'Bill'), ('Jones', 'Bob')}


lexsort can be a bit confusing the first time you use it because the order in which the keys are used to order the data starts with the last array passed. As you can see, last_name was used before first_name.

### Alternate Sort Algorithms

A stable sorting algorithm preserves the relative position of equal elements. This can be especially important in indirect sorts where the relative ordering is meaningful:

In [218]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])

In [219]:
key = np.array([2, 2, 1, 1 ,1])

In [220]:
indexer = key.argsort(kind = 'mergesort')

In [221]:
indexer

array([2, 3, 4, 0, 1], dtype=int64)

In [222]:
values.take(indexer)

array(['1:first', '1:second', '1:third', '2:first', '2:second'],
      dtype='<U8')

The only stable sort available is mergesort which has guaranteed O(n log n) performance (for complexity buffs), but its performance is on average worse than the default quicksort method.

![Array sorting methods](../../Pictures/Array%20sorting%20methods.png)

### numpy.searchsorted: Finding elements in a Sorted Array

searchsorted is an array method that performs a binary search on a sorted array, returning the location in the array where the value would need to be inserted to maintain sortedness:

In [232]:
arr = np.array([0, 1, 7, 12, 15])

In [238]:
arr.searchsorted(10)

3

As you might expect, you can also pass an array of values to get an array of indices back:

In [241]:
arr.searchsorted([0, 8, 11, 19])

array([0, 3, 3, 5], dtype=int64)

You might have noticed that searchsorted returned 0 for the 0 element. This is because the default behavior is to return the index at the left side of a group of equal values:

In [242]:
arr = np.array([0, 1, 0, 1, 0, 1, 1 ,1])

In [245]:
arr.searchsorted([1, 0])

array([5, 0], dtype=int64)

In [246]:
arr.searchsorted([0, 1], side = 'right')

array([5, 8], dtype=int64)

As another application of searchsorted, suppose we had an array of values between 0 and 10,000) and a separate array of “bucket edges” that we wanted to use to bin the data:

In [255]:
data = np.floor(np.random.uniform(0, 10000, size=50))

In [257]:
bins = np.array([0, 100, 1000, 5000, 10000])

In [258]:
data

array([ 377., 2136., 4386., 8924.,  913., 4195., 3059., 8652., 7159.,
       6876., 5851., 3758., 3373., 4892., 5660., 8132., 5768., 1978.,
       3317., 9451.,  909., 7226., 7118., 5165., 3940., 7618., 3018.,
       2316., 5541., 8260., 9800., 8679., 6085., 1611., 5469., 4872.,
       2816., 9154., 7483., 6136., 8855., 5892., 2842.,  256.,  651.,
       6884., 1612., 3854., 9927., 9034.])

To then get a labeling of which interval each data point belongs to (where 1 would mean the bucket [0, 100)), we can simply use searchsorted:

In [259]:
labesl = bins.searchsorted(data)

In [260]:
labesl

array([2, 3, 3, 4, 2, 3, 3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 4, 2, 4,
       4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 2,
       2, 4, 3, 3, 4, 4], dtype=int64)

This, combined with pandas’s groupby, can be used to easily bin data:

In [262]:
from pandas import Series

In [263]:
Series(data).groupby(labesl).mean()

2     621.200000
3    3220.833333
4    7437.000000
dtype: float64

Note that NumPy actually has a function digitize that computes this bin labelling 

In [264]:
np.digitize(data, bins)

array([2, 3, 3, 4, 2, 3, 3, 4, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 3, 4, 2, 4,
       4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 4, 3, 3, 4, 4, 4, 4, 4, 3, 2,
       2, 4, 3, 3, 4, 4], dtype=int64)

# NumPy Matrix Class

ompared with other languages for matrix operations and linear algebra, like MATLAB, Julia, and GAUSS, NumPy’s linear algebra syntax can often be quite verbose. One reason is that matrix multiplication requires using numpy.dot. Also NumPy’s indexing semantics are different, which makes porting code to Python less straightforward at times. Selecting a single row (e.g. X[1, :]) or column (e.g. X[:, 1]) from a 2D array yields a 1D array compared with a 2D array as in, say, MATLAB.

In [265]:
X = np.array([[ 8.82768214, 3.82222409, -1.14276475, 2.04411587],
            [ 3.82222409, 6.75272284, 0.83909108, 2.08293758],
            [-1.14276475, 0.83909108, 5.01690521, 0.79573241],
            [ 2.04411587, 2.08293758, 0.79573241, 6.24095859]])

In [266]:
X[:, 0] # one-dimensional

array([ 8.82768214,  3.82222409, -1.14276475,  2.04411587])

In [269]:
y = X[:, :1] # two-dimensional by slicing

In [268]:
X, y

(array([[ 8.82768214,  3.82222409, -1.14276475,  2.04411587],
        [ 3.82222409,  6.75272284,  0.83909108,  2.08293758],
        [-1.14276475,  0.83909108,  5.01690521,  0.79573241],
        [ 2.04411587,  2.08293758,  0.79573241,  6.24095859]]),
 array([[ 8.82768214],
        [ 3.82222409],
        [-1.14276475],
        [ 2.04411587]]))

In this case, the product $$y^t and X_y$$ would be expressed like so:

In [270]:
np.dot(y.T, np.dot(X, y))

array([[1195.46796121]])

To aid in writing code with a lot of matrix operations, NumPy has a matrix class which has modified indexing behavior to make it more MATLAB-like: single rows and columns come back two-dimensional and multiplication with * is matrix multiplication. The above operation with numpy.matrix would look like:

In [271]:
Xm = np.matrix(X)

In [272]:
ym = Xm[:, 0]

In [273]:
Xm

matrix([[ 8.82768214,  3.82222409, -1.14276475,  2.04411587],
        [ 3.82222409,  6.75272284,  0.83909108,  2.08293758],
        [-1.14276475,  0.83909108,  5.01690521,  0.79573241],
        [ 2.04411587,  2.08293758,  0.79573241,  6.24095859]])

In [274]:
ym

matrix([[ 8.82768214],
        [ 3.82222409],
        [-1.14276475],
        [ 2.04411587]])

In [275]:
ym.T * Xm * ym

matrix([[1195.46796121]])

matrix also has a special attribute I which returns the matrix inverse:

In [280]:
Xm.I 

matrix([[ 0.16847696, -0.09291247,  0.05894191, -0.03168704],
        [-0.09291247,  0.2181674 , -0.05198195, -0.03575436],
        [ 0.05894191, -0.05198195,  0.22633359, -0.03081416],
        [-0.03168704, -0.03575436, -0.03081416,  0.1864723 ]])

### Memory-mapped Files

A memory-mapped file is a method for treating potentially very large binary data on disk as an in-memory array. NumPy implements a memmap object that is ndarray-like, enabling small segments of a large file to be read and written without reading the whole array into memory. Additionally, a memmap has the same methods as an in-memory array and thus can be substituted into many algorithms where an ndarray would be expected.

To create a new memmap, use the function np.memmap and pass a file path, dtype, shape,and file mode:

In [294]:
mmap = np.memmap('mymmapp', dtype='float64', mode='w+', shape=(10000, 10000))

In [295]:
mmap

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Slicing a memmap returns views on the data on disk:

In [296]:
section = mmap[:5]

section

memmap([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

If you assign data to these, it will be buffered in memory (like a Python file object), but can be written to disk by calling flush:

In [297]:
section[:] = np.random.randn(5, 10000)

In [298]:
mmap.flush()

In [ ]:
mmap

memmap([[-1.38445117, -0.10637759,  2.37593791, ...,  0.24901834,
          1.71259071,  1.00888281],
        [-0.26655832, -0.73164203,  0.76765099, ...,  0.11378452,
         -0.93176745,  0.371228  ],
        [-0.27116823, -0.05992871, -0.89374337, ...,  1.07031855,
         -0.13303314,  0.14430261],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

In [ ]:
del mmap

Whenever a memory map falls out of scope and is garbage-collected, any changes will be flushed to disk also. When opening an existing memory map, you still have to specify the dtype and shape as the file is just a block of binary data with no metadata on disk:

In [302]:
mmap = np.memmap('mymmap', dtype='float64', shape=(10000, 10000))

In [303]:
mmap

memmap([[-1.38445117, -0.10637759,  2.37593791, ...,  0.24901834,
          1.71259071,  1.00888281],
        [-0.26655832, -0.73164203,  0.76765099, ...,  0.11378452,
         -0.93176745,  0.371228  ],
        [-0.27116823, -0.05992871, -0.89374337, ...,  1.07031855,
         -0.13303314,  0.14430261],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]])

Since a memory map is just an on-disk ndarray, there are no issues using a structured dtype as described above.

#### The Importance of Contiguous Memory

While the full extent of this topic is a bit outside the scope of this book, in some applications the memory layout of an array can significantly affect the speed of computations. This is based partly on performance differences having to do with the cache hierarchy of the CPU; operations accessing contiguous blocks of memory (for example, summing the rows of a C order array) will generally be the fastest because the memory subsystem will buffer the appropriate blocks of memory into the ultrafast L1 or L2 CPU cache. Also, certain code paths inside NumPy’s C codebase have been optimized for the contiguous case in which generic strided memory access can be avoided. To say that an array’s memory layout is contiguous means that the elements are stored in memory in the order that they appear in the array with respect to Fortran (column major) or C (row major) ordering. By default, NumPy arrays are created as C-contiguous or just simply contiguous. A column major array, such as the transpose of a C contiguous array, is thus said to be Fortran-contiguous. These properties can be explicitly checked via the flags attribute on the ndarray:

In [304]:
arr_c = np.ones ((1000, 1000), order = 'C')

arr_f = np.ones ((1000, 1000), order = 'F')

In [305]:
arr_c.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [306]:
arr_f.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [307]:
arr_f.flags.f_contiguous

True

In this example, summing the rows of these arrays should, in theory, be faster for arr_c than arr_f since the rows are contiguous in memory. Here I check for sure using %timeit in IPython:

In [308]:
%timeit arr_c.sum(1)

1.75 ms ± 135 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [309]:
%timeit arr_f.sum(1)

1.19 ms ± 67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


When looking to squeeze more performance out of NumPy, this is often a place to invest some effort. If you have an array that does not have the desired memory order, you can use copy and pass either 'C' or 'F':

In [310]:
arr_f.copy('C').flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

When constructing a view on an array, keep in mind that the result is not guaranteed to be contiguous:

In [311]:
arr_c[:50].flags.contiguous

True

In [312]:
arr_c[:, :50].flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

### Other Speed Options: Cython, f2py, C

In recent years, the Cython project ((http://cython.org) has become the tool of choice for many scientific Python programmers for implementing fast code that may need to interact with C or C++ libraries, but without having to write pure C code. You can think of Cython as Python with static types and the ability to interleave functions implemented in C into Python-like code. For example, a simple Cython function to sum the elements of a one-dimensional array might look like:

In [323]:
def sum_elem(ndarray([float64_t]) arr):

cdef Py_ssize_t i, n = len(arr)
cdef float64_t result = 0

for i in range(n):
    result += arr[i]

return result

SyntaxError: invalid syntax (3880720058.py, line 1)